In [2]:
import os
import sys

import numpy as np 
import open3d as o3d


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
# demo_icp_clouds = o3d.data.DemoICPPointClouds()
sigma = 0.005
src = o3d.io.read_point_cloud("src.ply")
dst = o3d.io.read_point_cloud(f"dst_noise_{sigma}.ply")
P = np.asarray(src.points)
Q = np.asarray(dst.points)

print(P)
print(Q)
print(P.shape)
print(Q.shape)

[[1.01953125 0.88671875 2.27726722]
 [1.04296875 0.88671875 2.27699804]
 [1.05859375 0.88671875 2.27722216]
 ...
 [1.26609075 2.37890625 2.26171875]
 [1.26171875 2.38015151 2.26171875]
 [1.26171875 2.37890625 2.26367211]]
[[0.28287324 1.02295592 1.13357777]
 [0.30509642 1.02689382 1.13579771]
 [0.31164948 1.0144455  1.14849564]
 ...
 [0.49383145 2.42138957 1.62567055]
 [0.49019162 2.42985608 1.62889702]
 [0.49414839 2.42393607 1.63102226]]
(198835, 3)
(198835, 3)


In [4]:
# Kabsch algorithm for unique random pairs

print(P.shape[0])
N_all = P.shape[0] - P.shape[0]%P.shape[1]
print(N_all) # 0 ~ 198833
stride = int(N_all/3)
print(stride)

# unique random 3 indices
SEED = 0 # deterministic
indices_random = np.random.default_rng(seed=SEED).choice(P.shape[0], N_all, replace=False).reshape((stride, P.shape[1]))
print(indices_random)
print(indices_random.shape)

# select from indices
P_selected = np.asmatrix(P[indices_random[0]])
# P_selected = np.asmatrix(P[0])
print(P_selected)
print(type(P_selected))
Q_selected = np.asmatrix(Q[indices_random[0]])
# Q_selected = np.asmatrix(Q[0])
print(Q_selected)
print(type(Q_selected))

print(P_selected.shape)
print(Q_selected.shape)

198835
198834
66278
[[ 78262  67929 134603]
 [160826  67523 189929]
 [158760 124440 183588]
 ...
 [ 34847   3286  14959]
 [  8146  61206  53642]
 [101630 126649 169133]]
(66278, 3)
[[2.16015625 1.72265625 1.86595058]
 [2.17357731 1.60546875 1.85546875]
 [2.69921875 2.25390625 1.07963181]]
<class 'numpy.matrix'>
[[1.46101863 1.74560496 1.61303415]
 [1.47555283 1.6296545  1.57219541]
 [2.34344063 2.35672117 1.39527359]]
<class 'numpy.matrix'>
(3, 3)
(3, 3)


In [5]:
def rigid_transform_3D(A, B, scale):

    assert len(A) == len(B)
    N = A.shape[0]

    # mean of each point cloud
    Am = np.mean(A, axis=0)
    Bm = np.mean(B, axis=0)

    # if len(Am.shape) == 1: Am = Am.reshape(1, N)
    # if len(Bm.shape) == 1: Bm = Bm.reshape(1, N)

    # centered point clouds
    Ac = A - np.tile(Am, (N, 1))
    Bc = B - np.tile(Bm, (N, 1))

    H = np.transpose(Bc) * Ac # NOTE: Kabsch; H = P^T \cdot Q
    if scale: H /= N

    """
    Based on rotation formula, optimal rotation R is

    R = sqrt(H^T \cdot H) \cdot H^-1

    since directly solving this is complicated and hard to handle edge cases (e.g., singular H), use SVD
    """

    U, S, Vt = np.linalg.svd(H) # NOTE: Kabsch; H = USV^T
    
    R = Vt.T * U.T # NOTE: Kabsch; R = V \cdot U^T

    # special reflection case
    if np.linalg.det(R) < 0:
        print("[DEBUG] Reflection detected")
        Vt[2, :] *= -1
        R = Vt.T * U.T
    
    if scale:
        Avar = np.var(A, axis=0).sum()
        c = 1 / (1 / Avar * np.sum(S)) # scale singular value
        t = -R.dot(Bm.T * c) + Am.T
        
    else:
        c = 1
        t = -R.dot(Bm.T) + Am.T

    return c, R, t


s, R, t = rigid_transform_3D(P_selected, Q_selected, False)
pose_from_kabsch = np.eye(4)
pose_from_kabsch[:3, :3] = R
pose_from_kabsch[0, 3] = t[0]
pose_from_kabsch[1, 3] = t[1]
pose_from_kabsch[2, 3] = t[2]
print(pose_from_kabsch)

[[ 0.85795064 -0.18121937  0.48070806  0.4466079 ]
 [ 0.02319935  0.94842963  0.31613767 -0.47497049]
 [-0.51320804 -0.2600784   0.81790998  1.75170697]
 [ 0.          0.          0.          1.        ]]


In [6]:
P_selected2 = (R * Q_selected.T) + np.tile(t, (1, 3))
P_selected2 = P_selected2.T
print(P_selected)
print(P_selected2)

[[2.16015625 1.72265625 1.86595058]
 [2.17357731 1.60546875 1.85546875]
 [2.69921875 2.25390625 1.07963181]]
[[2.15915085 1.72444852 1.86722304]
 [2.17300144 1.60190419 1.85651778]
 [2.70080002 2.25567854 1.07731032]]


In [7]:
# export point clouds

# np.savetxt("P_selected.txt", P_selected)
# np.savetxt("Q_selected.txt", Q_selected)

In [8]:
# TODO: check ordering

T = np.loadtxt("reg_p2l.txt")

q_h = np.ndarray((Q_selected[0].shape[0], Q_selected[0].shape[1]+1))
q_h[:, :3] = Q_selected[0]
q_h[:, 3] = 1.0
q_h = q_h.T

print(f"[DEBUG] np.linalg.inv(T) = \n{np.linalg.inv(T)}")
print(f"[DEBUG] pose_from_kabsch = \n{pose_from_kabsch}")
Tq = np.linalg.inv(T).dot(q_h).T[:, :3]
print(Tq)
print(P_selected[0])

[DEBUG] np.linalg.inv(T) = 
[[ 0.84035666 -0.14698139  0.52217257  0.34992254]
 [ 0.00650732  0.96478213  0.26252432 -0.39637089]
 [-0.54174376 -0.21667365  0.81183005  1.73064813]
 [ 0.          0.          0.          1.        ]]
[DEBUG] pose_from_kabsch = 
[[ 0.85795064 -0.18121937  0.48070806  0.4466079 ]
 [ 0.02319935  0.94842963  0.31613767 -0.47497049]
 [-0.51320804 -0.2600784   0.81790998  1.75170697]
 [ 0.          0.          0.          1.        ]]
[[2.16341002 1.72072559 1.87043339]]
[[2.16015625 1.72265625 1.86595058]]


In [9]:
for idx, indices in enumerate(indices_random):
    P_selected = np.asmatrix(P[indices])
    Q_selected = np.asmatrix(Q[indices])

    s, R, t = rigid_transform_3D(P_selected, Q_selected, False)
    pose_from_kabsch = np.eye(4)
    pose_from_kabsch[:3, :3] = R
    pose_from_kabsch[0, 3] = t[0]
    pose_from_kabsch[1, 3] = t[1]
    pose_from_kabsch[2, 3] = t[2]

    dir_result = os.path.abspath(f"sigma={sigma}")
    # print(dir_result)
    if not os.path.exists(dir_result):
        os.makedirs(dir_result)
    path_pose_from_kabsch = os.path.join(dir_result, f"pose_from_kabsch_{idx}.txt")
    # print(path_pose_from_kabsch)
    np.savetxt(path_pose_from_kabsch, pose_from_kabsch)
    
    

/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005/pose_from_kabsch_0.txt
[DEBUG] Reflection detected
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005/pose_from_kabsch_1.txt
[DEBUG] Reflection detected
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005/pose_from_kabsch_2.txt
[DEBUG] Reflection detected
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005/pose_from_kabsch_3.txt
[DEBUG] Reflection detected
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005
/root/Documents/Projects/augmented_safeguard/scripts/notebooks/sigma=0.005/pose_from_kabsch_4.txt
[DEBUG] Reflection dete